In [2]:
!pip install pinecone
! pip install langchain
! pip install tiktoken
! pip install pypdf
!pip install langchain-community
!pip install langchain-google-genai

### Import necesssary libraries

In [3]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
import pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [4]:
!mkdir pdfs

mkdir: cannot create directory ‘pdfs’: File exists


### Load the data and converts into chunks

In [5]:
loader = PyPDFDirectoryLoader('pdfs')

In [6]:
data = loader.load()

In [7]:
data[1]

Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'pdfs/Attention Is All You Need  by Ashish Vaswani .pdf', 'total_pages': 15, 'page': 1, 'page_label': '2'}, page_content='1 Introduction\nRecurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks\nin particular, have been firmly established as state of the art approaches in sequence modeling and\ntransduction problems such as language modeling and machine translation [ 35, 2, 5]. Numerous\nefforts have since continued to push the boundaries of recurrent language models and encoder-decoder\narchitectures [38, 24, 15].\nRecurrent models typically factor computation along the symbol positions 

In [8]:
data[0]

Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'pdfs/Attention Is All You Need  by Ashish Vaswani .pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.ed

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)

In [10]:
text_chunks = text_splitter.split_documents(data)

In [11]:
text_chunks[1]

Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'pdfs/Attention Is All You Need  by Ashish Vaswani .pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Łukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions')

In [12]:
text_chunks[0].page_content

'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu'

In [13]:
len(text_chunks)

90

### Initilize a client
Initialize a client connection to Pinecone and begin managing your data.

In [19]:
import os
import google.generativeai as genai
os.environ['GOOGLE_API_KEY'] = 'example_api_AIzaSyBlfoy7L0'
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

In [22]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os


# Pass the API key when initializing
embeddings = GoogleGenerativeAIEmbeddings(
    google_api_key=os.environ["GOOGLE_API_KEY"],
    model="models/embedding-001"
)
len(embeddings.embed_query("What's our relation?"))


768

In [36]:
from pinecone import Pinecone, ServerlessSpec

PINECONE_API_KEY="YOUR_API_KEY"

pc = Pinecone(api_key="API_KEY")

### Upsert data
Now that you are connected to Pinecone, the next critical step is to set up an index to store your data.

In [ ]:
# Create a serverless index
index_name = 'any_name'

pc.create_index(
    name=index_name,
    dimension=1024,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [ ]:
# Create vector embeddings
embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d['text'] for d in data],
    parameters={"input_type": "passage", "truncate": "END"}
)
print(embeddings[0])

In [ ]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for d, e in zip(data, embeddings):
    vectors.append({
        "id": d['id'],
        "values": e['values'],
        "metadata": {'text': d['text']}
    })

index.upsert(
    vectors=vectors,
    namespace="ns1"
)

In [ ]:
print(index.describe_index_stats())

### Query
Search through the data to find items that are semantically similar to a query vector.



In [ ]:
query = "Tell me about the tech company known as Apple."

embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    }
)

In [ ]:
results = index.query(
    namespace="ns1",
    vector=embedding[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)